In [ ]:
import codecs
import pandas as pd
import bs4
import requests
#Dùng time để truy cập các URL với startTime khác nhau
# Ví dụ: https://sports.theonion.com/?startTime=1585067580290
time = [1591289220194,1589395680468,1588268460533,1588271100468,
        1587140580419,1585163880765,1583849340543,1580220000317,
       1582814880415,1581523200230,1580212800340,1588360800834,1585067580290]
url = 'https://local.theonion.com/?startTime='
#Lấy file '.html' từ url đích
def get_page_content(url):
    page = requests.get(url,headers={"Accept-Language":"en-US"})
    return bs4.BeautifulSoup(page.text,"html.parser")
#Đọc file '.html' trong trường hợp HTTP secured
"""def DocFile(path):
    file=codecs.open(path,"r")
    file1=file.read()
    file1=str(file1)
    soup = bs4.BeautifulSoup(file1,'html.parser')
    return soup"""
#quét text html
def parse(soup):
    article = soup.findAll('div',class_="cw4lnv-11 dFCKPx")
    links = [art.find('a') for art in article]
    link = []
    #get links_article
    for l in links:
        link.append(l.get('href'))
    headlines = [head.find('h2',class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE").text for head in article]
    return link, headlines

#Chuyển tẽxt trên thành DataFrame
def get_df(link,headlines):
    papers = {"Article":link, "Headlines":headlines,"is_sarcasm":[1-1]*len(link)}
    data = pd.DataFrame(papers)
    return data
    #data.to_csv('sarcasm.csv', header = True, index = None)
    
#Đưa DF về csv
def get_csv(listdata):
    data = listdata[0]
    for i in range(1,len(listdata)):
        data = data.append(listdata[i])
    result = data.to_csv("sarcasm_onion2.csv",header = True, index = None)

#Bắt đầu quét nhiều URL với lists là tập các link cần quét    
def Sarcasm(lists):
    for arg in lists:
        soup = get_page_content(arg)
        link,headlines = parse(soup)
        data = get_df(link,headlines)
        
        yield data
#Tìm tất cả các link có thể có trong URL chủ
def get_link(soup):
    for b in soup:
        temp = str(b.get('href'))
        if temp.find('https')>-1 and temp.find('entertainment')<0:
            yield temp
#Tại URL:"https://www.theonion.com/" get link tất cả URL
#có className = "sc-1out364-0 hMndXN a1de4o-0 a1de4o-1 irneHt js_link"
#tức là các tab (đề mục) trên trang chủ.
links = list(get_link(get_page_content("https://www.theonion.com/").findAll('a',
            class_="sc-1out364-0 hMndXN a1de4o-0 a1de4o-1 irneHt js_link")))
#print(links)
#Với mỗi link được tạo ra sau khi get link
#tạo ra một url mới với mục đích crawl nhiều tên miền ở nhiều thời điểm khác nhau
#(kết hợp với list time)
lists = [string +'/?startTime='+ str(i) for i in time for string in links]
#print(lists)
#Tạo list với nhiều tên miền khác nhau.
#Sau đó tạo DF
listada = list(Sarcasm(lists))
#Chuyển DF về csv
result = get_csv(listada)

In [29]:
db = pd.read_csv("sarcasm_onion2.csv",encoding='utf-8')

In [30]:
db['is_sarcasm']

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: is_sarcasm, Length: 1000, dtype: int64

In [31]:
sacra = pd.read_csv("sarcasms.csv")

In [32]:
sacra['is_sarcasm']

0      1
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Name: is_sarcasm, Length: 1000, dtype: int64

In [33]:
sarcasm = sacra.append(db,sort = True)

In [38]:
sarcasm.to_csv("nlp_sarcasm.csv", index = None, header = True)

In [39]:
sarcasm['is_sarcasm']


0      1
1      1
2      1
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: is_sarcasm, Length: 2000, dtype: int64

In [41]:
z = pd.read_csv("nlp_sarcasm.csv")

In [42]:
z['is_sarcasm']

0       1
1       1
2       1
3       1
4       1
       ..
1995    0
1996    0
1997    0
1998    0
1999    0
Name: is_sarcasm, Length: 2000, dtype: int64